# change None to NaN

In [ ]:
import cv2
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [170]:
df2 = df.copy()
df2.fillna(value=np.nan, inplace=True)

## list NaNs tuples

In [171]:
# display(df.apply(np.nanmean, axis=1))
def nans_list(df):
    import math
    nans = []
    for i in np.arange(df.shape[0]):
        for j in np.arange(df.shape[1]):
            if math.isnan(df.iloc[i,j]):
                nans.append((i,j))
    return nans

In [174]:
nans = nans_list(df2)
print(f"NaNs left to deal with: {len(nans)}")

NaNs left to deal with: 840


## for points not at the edge, if NaN take the mean around them

In [175]:
persistent_nans = []
for point in nans:
    if point[0]>4 and point[0]<df2.shape[0]-5:
        df2.iloc[point[0], point[1]] = np.nanmean(df2.iloc[point[0]-5:point[0]+6,point[1] ])

print(f"NaNs left to deal with at the edges: {len(nans_list(df2))}")

NaNs left to deal with at the edges: 140


/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [176]:
display(nans_list(df2))

[(0, 0),
 (0, 1),
 (0, 10),
 (0, 11),
 (0, 14),
 (0, 15),
 (0, 28),
 (0, 29),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 28),
 (1, 29),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 10),
 (2, 11),
 (2, 28),
 (2, 29),
 (3, 0),
 (3, 1),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 14),
 (3, 15),
 (3, 28),
 (3, 29),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 14),
 (4, 15),
 (4, 28),
 (4, 29),
 (5, 0),
 (5, 1),
 (6, 0),
 (6, 1),
 (7, 0),
 (7, 1),
 (8, 0),
 (8, 1),
 (9, 0),
 (9, 1),
 (10, 0),
 (10, 1),
 (95, 0),
 (95, 1),
 (95, 2),
 (95, 3),
 (95, 10),
 (95, 11),
 (95, 12),
 (95, 13),
 (95, 20),
 (95, 21),
 (95, 26),
 (95, 27),
 (96, 0),
 (96, 1),
 (96, 6),
 (96, 7),
 (96, 8),
 (96, 9),
 (96, 10),
 (96, 11),
 (96, 14),
 (96, 15),
 (96, 18),
 (96, 19),
 (96, 20),
 (96, 21),
 (96, 24),
 (96, 25),
 (97, 0),
 (97, 1),
 (97, 4),
 (97, 5),
 (97, 6),
 (97, 7),
 (97, 8),
 (97, 

In [177]:
reverse_top = np.arange(0,15)[::-1]

In [178]:
for row in reverse_top:
    for col in np.arange(df2.shape[1]):
        if math.isnan(df2.iloc[row, col]):
            df2.iloc[row, col] = df2.iloc[row+1, col]

    

In [179]:
for row in np.arange(df2.shape[0]-10, df2.shape[0]):
    for col in np.arange(df2.shape[1]):
        if math.isnan(df2.iloc[row, col]):
            df2.iloc[row, col] = df2.iloc[row-1, col]


In [180]:
print(f"NaNs left to deal with: {len(nans_list(df2))}")

NaNs left to deal with: 0


## Anomaly detection and correction

In [181]:

def anomaly_detector(arr):
    array_mean = np.mean(arr)
    array_std = np.std(arr)
    mid_point = arr.iloc[int((arr.shape[0]+1)/2 - 1)]
    if np.abs(mid_point-array_mean)/array_std > 2:
#         print('anomaly handled')
        return array_mean, True
    return mid_point, False
    
   

In [182]:
iterations = 0
found_anomalies = True
while found_anomalies:
    iterations += 1
    found_anomalies = False
    for row in np.arange(5,df2.shape[0]-5):
        for col in np.arange(df2.shape[1]):
            result = anomaly_detector(df2.iloc[row-5:row+6, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
print(f"{iterations} iterations required")       

/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


21 iterations required


## anomalies on edges (top/bottom 5 rows)

In [183]:
def edge_anomaly_detector(arr, val):
    array_mean = np.mean(arr)
    array_std = np.std(arr)
#     mid_point = arr.iloc[int((arr.shape[0]+1)/2 - 1)]
    if np.abs(val-array_mean)/array_std > 2:
#         print('anomaly handled')
        return array_mean, True
    return val, False
    
   

In [184]:
iterations = 0
found_anomalies = True
while found_anomalies:
    iterations += 1
    found_anomalies = False
    for row in np.arange(5):
        for col in np.arange(df2.shape[1]):
            result = edge_anomaly_detector(df2.iloc[:5, col], df2.iloc[row, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
    for row in np.arange(df2.shape[0]):
        for col in np.arange(df2.shape[1]):
            result = edge_anomaly_detector(df2.iloc[df2.shape[0]-5:, col], df2.iloc[row, col])
            df2.iloc[row, col] = result[0]
            if result[1]:
                found_anomalies = True
print(f"{iterations} iterations required")    

/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/roee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


5 iterations required


In [185]:
display(df2)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1139.244000,211.000000,969.950000,469.000000,834.000000,352.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,540.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
1,1139.244000,211.000000,969.950000,493.000000,834.000000,493.000000,1126.0,305.0,1226.800000,516.000,...,1095.588479,982.963965,918.0,563.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
2,1139.244000,211.000000,969.950000,493.000000,850.800000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,793.0,493.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
3,1139.244000,211.000000,969.950000,493.000000,834.000000,375.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,563.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
4,1139.244000,211.000000,969.950000,481.000000,834.000000,352.000000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,540.0,1043.0,722.4,1113.094748,941.226804,993.200000,455.200000
5,1139.244000,211.000000,969.950000,481.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1095.588479,986.000000,793.0,399.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
6,1139.244000,211.000000,969.950000,487.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1047.300000,1004.600000,793.0,375.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
7,1139.244000,211.000000,969.950000,487.000000,918.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,834.0,493.0,1043.0,722.4,1123.418950,941.226804,993.200000,455.200000
8,1139.244000,211.000000,969.950000,487.000000,960.000000,361.200000,1126.0,305.0,1226.800000,514.872,...,1085.000000,986.000000,876.0,516.0,1043.0,722.4,1113.094748,950.134022,993.200000,455.200000
9,1139.244000,211.000000,969.950000,485.500000,936.555556,80.333333,1126.0,305.0,1226.800000,514.872,...,1095.588479,970.819824,834.0,469.0,1043.0,722.4,1113.094748,950.134022,993.200000,257.317708
